# Pointer

In [1]:
import json
import sys

from konfoo import *

## YAML Support

In [2]:
import oyaml as yaml

In [3]:
def to_yaml(container, *attributes, **options):
    flow_style = options.pop('flow_style', False)
    return yaml.safe_dump(container.view_fields(*attributes, **options), 
                          stream=sys.stdout, 
                          default_flow_style=flow_style)

## Define a Data Object

In [4]:
class DataObject(Structure):
    def __init__(self):
        super().__init__()
        self.size = Decimal(16)
        self.item = Pointer()
        self.index_fields()

In [5]:
data_object = DataObject()

In [6]:
data_object.to_list()

[('DataObject.size', 0), ('DataObject.item', '0x0')]

In [7]:
data_object.to_json()

'{"size": 0, "item": "0x0"}'

In [8]:
to_yaml(data_object, flow_style=False)

size: 0
item: '0x0'


In [9]:
data_object.to_list(nested=True)

[('DataObject.size', 0), ('DataObject.item', '0x0')]

In [10]:
data_object.to_json(nested=True)

'{"size": 0, "item": {"value": "0x0", "data": null}}'

In [11]:
to_yaml(data_object, nested=True, flow_style=False)

size: 0
item:
  value: '0x0'
  data: null


## Define a Data Object Pointer

In [12]:
class DataObjectPointer(Pointer):

    def __init__(self, address=None, byte_order=BYTEORDER):
        super().__init__(template=DataObject(),
                         address=address,
                         data_order=byte_order)

In [13]:
pointer = DataObjectPointer()

In [14]:
pointer.to_list()

[('DataObjectPointer.field', '0x0'),
 ('DataObjectPointer.data.size', 0),
 ('DataObjectPointer.data.item', '0x0')]

In [15]:
pointer.to_json()

'{"value": "0x0", "data": {"size": 0, "item": "0x0"}}'

In [16]:
to_yaml(pointer, flow_style=False)

value: '0x0'
data:
  size: 0
  item: '0x0'


In [17]:
pointer.to_list(nested=True)

[('DataObjectPointer.field', '0x0'),
 ('DataObjectPointer.data.size', 0),
 ('DataObjectPointer.data.item', '0x0')]

In [18]:
pointer.to_json(nested=True)

'{"value": "0x0", "data": {"size": 0, "item": {"value": "0x0", "data": null}}}'

In [19]:
to_yaml(pointer, nested=True, flow_style=False)

value: '0x0'
data:
  size: 0
  item:
    value: '0x0'
    data: null


## Nest Pointers

### Nested `pointer` with no `data object`.

In [20]:
pointer = Pointer(Pointer())

In [21]:
pointer.to_list()

[('Pointer.field', '0x0'), ('Pointer.data', '0x0')]

In [22]:
pointer.to_json()

'{"value": "0x0", "data": "0x0"}'

In [23]:
to_yaml(pointer, flow_style=False)

value: '0x0'
data: '0x0'


In [24]:
pointer.to_list(nested=True)

[('Pointer.field', '0x0'), ('Pointer.data', '0x0')]

In [25]:
pointer.to_json(nested=True)

'{"value": "0x0", "data": {"value": "0x0", "data": null}}'

In [26]:
to_yaml(pointer, nested=True, flow_style=False)

value: '0x0'
data:
  value: '0x0'
  data: null


### Nested `pointer` with an `data object`

In [27]:
pointer = Pointer(Pointer(Byte()))

In [28]:
pointer.to_list()

[('Pointer.field', '0x0'), ('Pointer.data', '0x0')]

In [29]:
pointer.to_json()

'{"value": "0x0", "data": "0x0"}'

In [30]:
to_yaml(pointer, flow_style=False)

value: '0x0'
data: '0x0'


In [31]:
pointer.to_list(nested=True)

[('Pointer.field', '0x0'),
 ('Pointer.data', '0x0'),
 ('Pointer.data.data', '0x0')]

In [32]:
pointer.to_json(nested=True)

'{"value": "0x0", "data": {"value": "0x0", "data": "0x0"}}'

In [33]:
to_yaml(pointer, nested=True, flow_style=False)

value: '0x0'
data:
  value: '0x0'
  data: '0x0'


## Access the Data Object

In [34]:
pointer = Pointer(Pointer(Byte()))

In [35]:
pointer.data

Pointer(index=Index(byte=0, bit=0, address=0, base_address=0, update=False), alignment=Alignment(byte_size=4, bit_offset=0), bit_size=32, value='0x0')

In [36]:
pointer.data.data

Byte(index=Index(byte=0, bit=0, address=0, base_address=0, update=False), alignment=Alignment(byte_size=1, bit_offset=0), bit_size=8, value='0x0')

## Address of the Data Object

In [37]:
pointer.address

0

## Byte Order of the Data Object

In [38]:
pointer.data_byte_order

Byteorder.little = 'little'

In [39]:
pointer.data_byte_order = 'big'

In [40]:
pointer.data_byte_order

Byteorder.big = 'big'

## Byte Stream for the Data Object

In [41]:
pointer.bytestream

''

In [42]:
pointer.bytestream = '000000000000'

In [43]:
pointer.bytestream

'000000000000'

## Initialize a Pointer

In [44]:
pointer = Pointer(
    Structure(
        size=Decimal(16),
        item=Pointer(String(14))))

In [45]:
pointer.to_list(nested=True)

[('Pointer.field', '0x0'),
 ('Pointer.data.size', 0),
 ('Pointer.data.item', '0x0'),
 ('Pointer.data.item.data', '')]

In [46]:
pointer.to_json(nested=True)

'{"value": "0x0", "data": {"size": 0, "item": {"value": "0x0", "data": ""}}}'

In [47]:
to_yaml(pointer, nested=True, flow_style=False)

value: '0x0'
data:
  size: 0
  item:
    value: '0x0'
    data: ''


In [48]:
pointer.initialize_fields({
    'value': 0x1,
    'data': {
        'size': 14,
        'item': {'value': 0x10,
                 'data': 'Konfoo is Fun'}}})

In [49]:
pointer.to_list(nested=True)

[('Pointer.field', '0x1'),
 ('Pointer.data.size', 14),
 ('Pointer.data.item', '0x10'),
 ('Pointer.data.item.data', 'Konfoo is Fun')]

In [50]:
pointer.to_json(nested=True)

'{"value": "0x1", "data": {"size": 14, "item": {"value": "0x10", "data": "Konfoo is Fun"}}}'

In [51]:
to_yaml(pointer, nested=True, flow_style=False)

value: '0x1'
data:
  size: 14
  item:
    value: '0x10'
    data: Konfoo is Fun


## Display a Pointer

In [52]:
pointer = Pointer(
    Structure(
        size=Decimal(16),
        item=Pointer(String())))

In [53]:
pointer.index_fields()

Index(byte=4, bit=0, address=4, base_address=0, update=False)

In [54]:
pointer

Pointer(index=Index(byte=0, bit=0, address=0, base_address=0, update=False), alignment=Alignment(byte_size=4, bit_offset=0), bit_size=32, value='0x0')

## Display the Data Object

In [55]:
pointer.data

Structure([('size',
            Decimal(index=Index(byte=0, bit=0, address=0, base_address=0, update=False), alignment=Alignment(byte_size=2, bit_offset=0), bit_size=16, value=0)),
           ('item',
            Pointer(index=Index(byte=2, bit=0, address=2, base_address=0, update=False), alignment=Alignment(byte_size=4, bit_offset=0), bit_size=32, value='0x0'))])

## Metadata of a Pointer

In [56]:
pointer.describe()

OrderedDict([('address', 0),
             ('alignment', [4, 0]),
             ('class', 'Pointer'),
             ('index', [0, 0]),
             ('max', 4294967295),
             ('min', 0),
             ('name', 'Pointer'),
             ('order', 'auto'),
             ('signed', False),
             ('size', 32),
             ('type', 'Pointer'),
             ('value', '0x0'),
             ('member',
              [OrderedDict([('class', 'Structure'),
                            ('name', 'data'),
                            ('size', 2),
                            ('type', 'Structure'),
                            ('member',
                             [OrderedDict([('address', 0),
                                           ('alignment', [2, 0]),
                                           ('class', 'Decimal16'),
                                           ('index', [0, 0]),
                                           ('max', 65535),
                                           ('min', 0)

In [57]:
json.dump(pointer.describe(), sys.stdout, indent=2)

{
  "address": 0,
  "alignment": [
    4,
    0
  ],
  "class": "Pointer",
  "index": [
    0,
    0
  ],
  "max": 4294967295,
  "min": 0,
  "name": "Pointer",
  "order": "auto",
  "signed": false,
  "size": 32,
  "type": "Pointer",
  "value": "0x0",
  "member": [
    {
      "class": "Structure",
      "name": "data",
      "size": 2,
      "type": "Structure",
      "member": [
        {
          "address": 0,
          "alignment": [
            2,
            0
          ],
          "class": "Decimal16",
          "index": [
            0,
            0
          ],
          "max": 65535,
          "min": 0,
          "name": "size",
          "order": "auto",
          "signed": false,
          "size": 16,
          "type": "Field",
          "value": 0
        },
        {
          "address": 2,
          "alignment": [
            4,
            0
          ],
          "class": "Pointer",
          "index": [
            2,
            0
          ],
          "max": 42949

In [58]:
d3flare_json(pointer.describe(), sys.stdout, indent=2)

{
  "class": "Pointer",
  "name": "Pointer",
  "children": [
    {
      "class": "Pointer",
      "name": "*Pointer",
      "size": 32,
      "value": "0x0"
    },
    {
      "class": "Structure",
      "name": "data",
      "children": [
        {
          "class": "Decimal16",
          "name": "size",
          "size": 16,
          "value": 0
        },
        {
          "class": "Pointer",
          "name": "item",
          "children": [
            {
              "class": "Pointer",
              "name": "*item",
              "size": 32,
              "value": "0x0"
            },
            {
              "class": "String",
              "name": "data",
              "size": 0,
              "value": ""
            }
          ]
        }
      ]
    }
  ]
}

## Size of the Data Object

In [59]:
pointer.data_size

6

## Indexing

In [60]:
pointer = Pointer(
    Structure(
        size=Decimal(16),
        item=Pointer(String())))

In [61]:
pointer.to_list('index', nested=True)

[('Pointer.field',
  Index(byte=0, bit=0, address=0, base_address=0, update=False)),
 ('Pointer.data.size',
  Index(byte=0, bit=0, address=0, base_address=0, update=False)),
 ('Pointer.data.item',
  Index(byte=0, bit=0, address=0, base_address=0, update=False)),
 ('Pointer.data.item.data',
  Index(byte=0, bit=0, address=0, base_address=0, update=False))]

In [62]:
pointer.to_json('index', nested=True)

'{"value": [0, 0, 0, 0, false], "data": {"size": [0, 0, 0, 0, false], "item": {"value": [0, 0, 0, 0, false], "data": [0, 0, 0, 0, false]}}}'

In [63]:
pointer.index_fields(index=Index())

Index(byte=4, bit=0, address=4, base_address=0, update=False)

In [64]:
pointer.index_fields()

Index(byte=4, bit=0, address=4, base_address=0, update=False)

In [65]:
pointer.to_list('index', nested=True)

[('Pointer.field',
  Index(byte=0, bit=0, address=0, base_address=0, update=False)),
 ('Pointer.data.size',
  Index(byte=0, bit=0, address=0, base_address=0, update=False)),
 ('Pointer.data.item',
  Index(byte=2, bit=0, address=2, base_address=0, update=False)),
 ('Pointer.data.item.data',
  Index(byte=0, bit=0, address=0, base_address=0, update=False))]

In [66]:
pointer.to_json('index', nested=True)

'{"value": [0, 0, 0, 0, false], "data": {"size": [0, 0, 0, 0, false], "item": {"value": [2, 0, 2, 0, false], "data": [0, 0, 0, 0, false]}}}'

### Index the Pointer Field

In [67]:
pointer = Pointer(
    Structure(
        size=Decimal(16),
        item=Pointer(String())))

### Index the Data Object

In [68]:
pointer = Pointer(
    Structure(
        size=Decimal(16),
        item=Pointer(String())))

In [69]:
pointer.data.to_list('index', nested=True)

[('Structure.size',
  Index(byte=0, bit=0, address=0, base_address=0, update=False)),
 ('Structure.item',
  Index(byte=0, bit=0, address=0, base_address=0, update=False)),
 ('Structure.item.data',
  Index(byte=0, bit=0, address=0, base_address=0, update=False))]

In [70]:
pointer.data.to_json('index', nested=True)

'{"size": [0, 0, 0, 0, false], "item": {"value": [0, 0, 0, 0, false], "data": [0, 0, 0, 0, false]}}'

In [71]:
pointer.index_data()

In [72]:
pointer.data.to_list('index', nested=True)

[('Structure.size',
  Index(byte=0, bit=0, address=0, base_address=0, update=False)),
 ('Structure.item',
  Index(byte=2, bit=0, address=2, base_address=0, update=False)),
 ('Structure.item.data',
  Index(byte=0, bit=0, address=0, base_address=0, update=False))]

In [73]:
pointer.data.to_json('index', nested=True)

'{"size": [0, 0, 0, 0, false], "item": {"value": [2, 0, 2, 0, false], "data": [0, 0, 0, 0, false]}}'

## De-Serializing

In [74]:
pointer.deserialize(bytes.fromhex('00000000f00f00'))

Index(byte=4, bit=0, address=4, base_address=0, update=False)

In [75]:
pointer.to_list()

[('Pointer.field', '0x0'),
 ('Pointer.data.size', 0),
 ('Pointer.data.item', '0x0')]

In [76]:
pointer.to_json()

'{"value": "0x0", "data": {"size": 0, "item": "0x0"}}'

In [77]:
to_yaml(pointer, flow_style=False)

value: '0x0'
data:
  size: 0
  item: '0x0'


## Serializing

In [78]:
bytestream = bytearray()
bytestream.hex()

''

In [79]:
pointer.serialize(bytestream)

Index(byte=4, bit=0, address=4, base_address=0, update=False)

In [80]:
bytes(pointer).hex()

'00000000'

In [81]:
bytestream.hex()

'00000000'

## Pointer Members

### Access the Data Object of a Pointer Field

In [82]:
pointer.data

Structure([('size',
            Decimal(index=Index(byte=0, bit=0, address=0, base_address=0, update=False), alignment=Alignment(byte_size=2, bit_offset=0), bit_size=16, value=0)),
           ('item',
            Pointer(index=Index(byte=2, bit=0, address=2, base_address=0, update=False), alignment=Alignment(byte_size=4, bit_offset=0), bit_size=32, value='0x0'))])

### Access the Attributes of a Pointer Field

In [83]:
pointer.name

'Pointer32'

In [84]:
pointer.value

'0x0'

In [85]:
pointer.bit_size

32

In [86]:
pointer.alignment

Alignment(byte_size=4, bit_offset=0)

In [87]:
pointer.alignment.byte_size

4

In [88]:
pointer.alignment.bit_offset

0

In [89]:
pointer.byte_order

Byteorder.auto = 'auto'

In [90]:
pointer.byte_order.value

'auto'

In [91]:
pointer.index

Index(byte=0, bit=0, address=0, base_address=0, update=False)

In [92]:
pointer.index.byte

0

In [93]:
pointer.index.bit

0

In [94]:
pointer.index.address

0

In [95]:
pointer.index.base_address

0

In [96]:
pointer.index.update

False

## Pointer Fields

### View Field Attributes

In [97]:
pointer.view_fields()

OrderedDict([('value', '0x0'),
             ('data', OrderedDict([('size', 0), ('item', '0x0')]))])

In [98]:
pointer.view_fields(nested=True)

OrderedDict([('value', '0x0'),
             ('data',
              OrderedDict([('size', 0),
                           ('item',
                            OrderedDict([('value', '0x0'), ('data', '')]))]))])

In [99]:
pointer.view_fields('name', 'bit_size', 'value', 'index', 'alignment', 'byte_order.name')

OrderedDict([('name', 'Pointer32'),
             ('bit_size', 32),
             ('value', '0x0'),
             ('index',
              Index(byte=0, bit=0, address=0, base_address=0, update=False)),
             ('alignment', Alignment(byte_size=4, bit_offset=0)),
             ('byte_order.name', 'auto'),
             ('data',
              OrderedDict([('size',
                            {'name': 'Decimal16',
                             'bit_size': 16,
                             'value': 0,
                             'index': Index(byte=0, bit=0, address=0, base_address=0, update=False),
                             'alignment': Alignment(byte_size=2, bit_offset=0),
                             'byte_order.name': 'auto'}),
                           ('item',
                            {'name': 'Pointer32',
                             'bit_size': 32,
                             'value': '0x0',
                             'index': Index(byte=2, bit=0, address=2, base_address=0

### View as a JSON string

In [100]:
pointer.to_json()

'{"value": "0x0", "data": {"size": 0, "item": "0x0"}}'

In [101]:
print(pointer.to_json(indent=2))

{
  "value": "0x0",
  "data": {
    "size": 0,
    "item": "0x0"
  }
}


In [102]:
pointer.to_json(nested=True)

'{"value": "0x0", "data": {"size": 0, "item": {"value": "0x0", "data": ""}}}'

In [103]:
print(pointer.to_json(nested=True, indent=2))

{
  "value": "0x0",
  "data": {
    "size": 0,
    "item": {
      "value": "0x0",
      "data": ""
    }
  }
}


In [104]:
pointer.to_json('name', 'bit_size', 'value', 'index', 'alignment', 'byte_order')

'{"name": "Pointer32", "bit_size": 32, "value": "0x0", "index": [0, 0, 0, 0, false], "alignment": [4, 0], "byte_order": "auto", "data": {"size": {"name": "Decimal16", "bit_size": 16, "value": 0, "index": [0, 0, 0, 0, false], "alignment": [2, 0], "byte_order": "auto"}, "item": {"name": "Pointer32", "bit_size": 32, "value": "0x0", "index": [2, 0, 2, 0, false], "alignment": [4, 0], "byte_order": "auto"}}}'

In [105]:
print(pointer.to_json('name', 'bit_size', 'value', 'index', 'alignment', 'byte_order', indent=2))

{
  "name": "Pointer32",
  "bit_size": 32,
  "value": "0x0",
  "index": [
    0,
    0,
    0,
    0,
    false
  ],
  "alignment": [
    4,
    0
  ],
  "byte_order": "auto",
  "data": {
    "size": {
      "name": "Decimal16",
      "bit_size": 16,
      "value": 0,
      "index": [
        0,
        0,
        0,
        0,
        false
      ],
      "alignment": [
        2,
        0
      ],
      "byte_order": "auto"
    },
    "item": {
      "name": "Pointer32",
      "bit_size": 32,
      "value": "0x0",
      "index": [
        2,
        0,
        2,
        0,
        false
      ],
      "alignment": [
        4,
        0
      ],
      "byte_order": "auto"
    }
  }
}


### List the Field Items

In [106]:
pointer.field_items()

[('field',
  Pointer(index=Index(byte=0, bit=0, address=0, base_address=0, update=False), alignment=Alignment(byte_size=4, bit_offset=0), bit_size=32, value='0x0')),
 ('data.size',
  Decimal(index=Index(byte=0, bit=0, address=0, base_address=0, update=False), alignment=Alignment(byte_size=2, bit_offset=0), bit_size=16, value=0)),
 ('data.item',
  Pointer(index=Index(byte=2, bit=0, address=2, base_address=0, update=False), alignment=Alignment(byte_size=4, bit_offset=0), bit_size=32, value='0x0'))]

### List the Field Attributes

In [107]:
pointer.to_list()

[('Pointer.field', '0x0'),
 ('Pointer.data.size', 0),
 ('Pointer.data.item', '0x0')]

In [108]:
pointer.to_list(nested=True)

[('Pointer.field', '0x0'),
 ('Pointer.data.size', 0),
 ('Pointer.data.item', '0x0'),
 ('Pointer.data.item.data', '')]

In [109]:
pointer.to_list('name', 'bit_size', 'value', 'index', 'alignment', 'byte_order')

[('Pointer.field',
  ('Pointer32',
   32,
   '0x0',
   Index(byte=0, bit=0, address=0, base_address=0, update=False),
   Alignment(byte_size=4, bit_offset=0),
   Byteorder.auto = 'auto')),
 ('Pointer.data.size',
  ('Decimal16',
   16,
   0,
   Index(byte=0, bit=0, address=0, base_address=0, update=False),
   Alignment(byte_size=2, bit_offset=0),
   Byteorder.auto = 'auto')),
 ('Pointer.data.item',
  ('Pointer32',
   32,
   '0x0',
   Index(byte=2, bit=0, address=2, base_address=0, update=False),
   Alignment(byte_size=4, bit_offset=0),
   Byteorder.auto = 'auto'))]

In [110]:
pointer.to_dict()

OrderedDict([('Pointer',
              OrderedDict([('field', '0x0'),
                           ('data.size', 0),
                           ('data.item', '0x0')]))])

In [111]:
pointer.to_dict(nested=True)

OrderedDict([('Pointer',
              OrderedDict([('field', '0x0'),
                           ('data.size', 0),
                           ('data.item', '0x0'),
                           ('data.item.data', '')]))])

In [112]:
pointer.to_dict('name', 'bit_size', 'value', 'index', 'alignment', 'byte_order')

OrderedDict([('Pointer',
              OrderedDict([('field',
                            ('Pointer32',
                             32,
                             '0x0',
                             Index(byte=0, bit=0, address=0, base_address=0, update=False),
                             Alignment(byte_size=4, bit_offset=0),
                             Byteorder.auto = 'auto')),
                           ('data.size',
                            ('Decimal16',
                             16,
                             0,
                             Index(byte=0, bit=0, address=0, base_address=0, update=False),
                             Alignment(byte_size=2, bit_offset=0),
                             Byteorder.auto = 'auto')),
                           ('data.item',
                            ('Pointer32',
                             32,
                             '0x0',
                             Index(byte=2, bit=0, address=2, base_address=0, update=False),
     

### Save the Field Attributes to an `.ini` File

In [113]:
pointer.save('Pointer.ini', nested=True)

### Load the Field Value from an `.ini` File

In [114]:
pointer.load('Pointer.ini', nested=True)

[Pointer]
Pointer.field = 0x0
Pointer.data.size = 0
Pointer.data.item = 0x0
Pointer.data.item.data = 
